In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import CSVLogger, SaveModelCallback
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [0]:
#see the folders available
import os
os.listdir(os.getcwd())

['.config', 'sample_data']

In [0]:
os.mkdir('data')
%cd data

/content/data


In [0]:
#get the data from dropbox link
!wget https://www.dropbox.com/s/6kltw0kqynlijxv/widsdatathon2019.zip

--2019-02-25 09:37:38--  https://www.dropbox.com/s/6kltw0kqynlijxv/widsdatathon2019.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6kltw0kqynlijxv/widsdatathon2019.zip [following]
--2019-02-25 09:37:38--  https://www.dropbox.com/s/raw/6kltw0kqynlijxv/widsdatathon2019.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc98693ce806184946837db8e10d.dl.dropboxusercontent.com/cd/0/inline/Ab8RoQ_--2mQbbw3sK0psFNBElJtuo58kizP_B6kSnVIF7ngnaPk6OjBOTj4pLnCHjQxOwVcbLuQdpfOBnU7zr00IGR0USQPAjsFVQIltOMUNA/file# [following]
--2019-02-25 09:37:39--  https://uc98693ce806184946837db8e10d.dl.dropboxusercontent.com/cd/0/inline/Ab8RoQ_--2mQbbw3sK0psFNBElJtuo58kizP_B6kSnVIF7ngnaPk6OjBOTj4pLnCHjQxOwVcbLuQdpfOBnU7zr00IGR0USQPAjsFVQIltOMUNA/

In [0]:
#unzipping the data downloaded above
!unzip widsdatathon2019.zip

Archive:  widsdatathon2019.zip
  inflating: train_images.zip        
  inflating: traininglabels.csv      
  inflating: SampleSubmission.csv    
  inflating: leaderboard_holdout_data.zip  
  inflating: leaderboard_test_data.zip  


In [0]:
!unzip train_images.zip   
!unzip leaderboard_holdout_data.zip
!unzip leaderboard_test_data.zip

Archive:  train_images.zip
   creating: train_images/
  inflating: train_images/img_068062017.jpg  
  inflating: train_images/img_013962018.jpg  
  inflating: train_images/img_029652018.jpg  
  inflating: train_images/img_018402017.jpg  
  inflating: train_images/img_090232018.jpg  
  inflating: train_images/img_111972018.jpg  
  inflating: train_images/img_087662017.jpg  
  inflating: train_images/img_085292018.jpg  
  inflating: train_images/img_046872017.jpg  
  inflating: train_images/img_084512018.jpg  
  inflating: train_images/img_021842018.jpg  
  inflating: train_images/img_018502017.jpg  
  inflating: train_images/img_087762017.jpg  
  inflating: train_images/img_111872018.jpg  
  inflating: train_images/img_085392018.jpg  
  inflating: train_images/img_059332018.jpg  
  inflating: train_images/img_068162017.jpg  
  inflating: train_images/img_084412018.jpg  
  inflating: train_images/img_093042017.jpg  
  inflating: train_images/img_079162018.jpg  
  inflating: train_images/

In [0]:
%cd ..
os.listdir(os.getcwd())

/content


['.config', 'data', 'sample_data']

In [0]:
path = Path('data/')
path.ls()

[PosixPath('data/leaderboard_holdout_data'),
 PosixPath('data/leaderboard_holdout_data.zip'),
 PosixPath('data/leaderboard_test_data'),
 PosixPath('data/SampleSubmission.csv'),
 PosixPath('data/traininglabels.csv'),
 PosixPath('data/leaderboard_test_data.zip'),
 PosixPath('data/train_images.zip'),
 PosixPath('data/train_images'),
 PosixPath('data/widsdatathon2019.zip')]

In [0]:
train = Path('data/train_images')
test = Path('data/leaderboard_test_data')
lb_test = Path('data/leaderboard_holdout_data')
sample_sub = Path('data/SampleSubmission.csv')
train.ls()[:5]

[PosixPath('data/train_images/img_075442017.jpg'),
 PosixPath('data/train_images/img_025832017.jpg'),
 PosixPath('data/train_images/img_002562017.jpg'),
 PosixPath('data/train_images/img_023292018.jpg'),
 PosixPath('data/train_images/img_073492018.jpg')]

In [0]:
test_imgs = [i for i in test.iterdir()]
hold_imgs = [i for i in lb_test.iterdir()]
combined_test = test_imgs + hold_imgs
len(combined_test)

6534

In [0]:
df = pd.read_csv('data/traininglabels.csv')
df.head()

,image_id,has_oilpalm,score
0,img_000002017.jpg,0,0.7895
1,img_000012017.jpg,0,1.0000
2,img_000022017.jpg,0,1.0000
3,img_000072017.jpg,0,1.0000
4,img_000082017.jpg,0,1.0000


In [0]:
from sklearn.model_selection import KFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1,2], [3, 4], [1, 2], [3, 4],[1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
kf = KFold(n_splits=5)
kf.get_n_splits(df.image_id)

5

In [0]:
for i, (train_index, test_index) in enumerate(kf.split(df.image_id)):
#   print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = df.image_id[train_index], df.image_id[test_index]
  y_train, y_test = df.has_oilpalm[train_index], df.has_oilpalm[test_index]
  
  src = (ImageItemList.from_df(df, path, folder='train_images')
    .split_by_idx(valid_idx = test_index)
    .label_from_df('has_oilpalm'))
  
  data = (src.transform(get_transforms(flip_vert=True, max_zoom=1.1), size=224)
           .databunch(bs=32)
           .normalize(imagenet_stats))
  
  learn = create_cnn(data, models.resnet50, metrics=[accuracy, error_rate], 
                  callback_fns=[ShowGraph, SaveModelCallback])
  
#   learn.lr_find()
#   learn.recorder.plot()
  
  learn.fit_one_cycle(5, 1e-3)
  
  from sklearn.metrics import roc_auc_score
  def auc_score(y_score,y_true):
      return torch.tensor(roc_auc_score(y_true,y_score[:,1]))
  probs,val_labels = learn.get_preds(ds_type=DatasetType.Valid) 
  print('Accuracy',accuracy(probs,val_labels)),
  print('Error Rate', error_rate(probs, val_labels))
  print('AUC', auc_score(probs,val_labels))
  
  learn.save(f"fold_{i}")

Accuracy tensor(0.9938)
Error Rate tensor(0.0062)
AUC tensor(0.9990)


In [0]:
all_preds = []
for i in range(5):
  learn.load(f"fold_{i}")
  preds, _ = learn.get_preds()
  all_preds.append(np.array(preds))
  
avg_preds = np.mean(np.array(all_preds), axis=0)
print(avg_preds)
len(avg_preds)

[[9.998270e-01 1.730275e-04]
 [9.955762e-01 4.423839e-03]
 [9.999696e-01 3.040575e-05]
 [9.999488e-01 5.124578e-05]
 ...
 [9.993811e-01 6.189478e-04]
 [9.997351e-01 2.649021e-04]
 [3.947654e-01 6.052346e-01]
 [9.997615e-01 2.384750e-04]]


3048

In [0]:
data_test = (ImageItemList.from_df(df, path, folder='train_images')
              .split_by_idx(valid_idx = test_index)
              .label_from_df('has_oilpalm')
              .transform(get_transforms(flip_vert=True, max_zoom=1.1), size=224)
               .add_test(combined_test)
              .databunch(bs=32)
              .normalize(imagenet_stats))

In [0]:
learn.data = data_test
learn.validate()

[0.018070033, tensor(0.9938), tensor(0.0062)]

In [0]:
_, preds = learn.get_preds(ds_type=DatasetType.Test)

In [0]:
print('Accuracy',accuracy(probs,val_labels)),
print('Error Rate', error_rate(probs, val_labels))
print('AUC', auc_score(probs,val_labels))

Accuracy tensor(0.9938)
Error Rate tensor(0.0062)
AUC tensor(0.9990)


In [0]:
ids = np.array([f.name for f in (combined_test)]);

In [0]:
submission = pd.DataFrame(np.stack([ids, _[:,1]], axis=1), columns=pd.read_csv(sample_sub).columns)

In [0]:
submission.to_csv(path/'cv_submission.csv', index=False)